#### Train the Model

We used a simple yet powerful Random Forest Classifier to identify speakers based on their voice features.

In [1]:
# Import libraries

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import joblib
import os

#### **Load processed audio CSV**

In [2]:
csv_path = r"C:\users\LENOVO\Documents\formative2-mlp\Audios\processed_audio_features.csv"
df = pd.read_csv(csv_path)
print(f" Loaded data: {df.shape[0]} rows, {df.shape[1]} columns")
print(" Audio features loaded successfully!")
print(df.head())

 Loaded data: 32 rows, 69 columns
 Audio features loaded successfully!
                                    sample_id person  phrase  \
0      Egide_Confirm_Transaction.WAV_original  Egide     NaN   
1      Egide_Confirm_Transaction.WAV_pitch_up  Egide     NaN   
2  Egide_Confirm_Transaction.WAV_time_stretch  Egide     NaN   
3         Egide_Confirm_Transaction.WAV_noise  Egide     NaN   
4              Egide_Yes_Approve.WAV_original  Egide     NaN   

        augmentation  sample_rate  mfcc1_mean  mfcc1_std  mfcc1_min  \
0           Original        44100  -357.26420  78.297120 -565.46690   
1     Pitch Shift +2        44100  -382.09753  78.630370 -559.86890   
2  Time Stretch 1.1x        44100  -390.32140  75.748820 -571.45750   
3   Background Noise        44100  -158.61511  39.616135 -262.42654   
4           Original        44100  -396.49390  83.952850 -559.94037   

   mfcc1_max  mfcc2_mean  ...  rolloff_min   rolloff_max  energy_mean  \
0 -219.59917   152.54172  ...  1076.660156  

#### **Fill missing 'person' from 'sample_id'**

In [3]:
if df['person'].isna().all():
    print(" 'person' column empty. Extracting from 'sample_id'...")
    df['person'] = df['sample_id'].apply(lambda x: x.split('_')[0])

# Drop rows with missing 'person'
df = df.dropna(subset=['person'])
print(f" Data after filling 'person': {df.shape[0]} rows")

 Data after filling 'person': 32 rows


#### **Prepare features (X) and labels (y)**

In [4]:
exclude_cols = ['sample_id', 'person', 'phrase', 'augmentation', 'sample_rate']
X = df.drop(columns=[col for col in exclude_cols if col in df.columns])
y = df['person']

print(f"Features shape: {X.shape}")
print(f"Number of classes: {len(y.unique())}")

Features shape: (32, 64)
Number of classes: 4


#### **Split Data for Training and Testing**

We divide the dataset into training (80%) and testing (20%). This helps evaluate how well your model generalizes.

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f"Training samples: {X_train.shape[0]}, Testing samples: {X_test.shape[0]}")

Training samples: 25, Testing samples: 7


#### **Train Random Forest Classifier**

In [6]:
model = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    max_depth=None,
    n_jobs=-1
)

model.fit(X_train, y_train)
print(" Model training complete!")

 Model training complete!


#### **Evaluate on test set**

In [7]:
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Test accuracy: {acc*100:.2f}%")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Test accuracy: 85.71%

Classification Report:
              precision    recall  f1-score   support

       Egide       1.00      1.00      1.00         2
          JD       0.50      1.00      0.67         1
      Mariam       1.00      0.50      0.67         2
      Noella       1.00      1.00      1.00         2

    accuracy                           0.86         7
   macro avg       0.88      0.88      0.83         7
weighted avg       0.93      0.86      0.86         7



### **Step 8: Save model and metadata**

In [8]:
save_dir = r"C:\users\LENOVO\Documents\formative2-mlp\Audios"
os.makedirs(save_dir, exist_ok=True)

joblib.dump(model, os.path.join(save_dir, 'audio_model.pkl'))
joblib.dump(X.columns.tolist(), os.path.join(save_dir, 'feature_names.pkl'))
joblib.dump(model.classes_, os.path.join(save_dir, 'class_names.pkl'))

print(f" Model and metadata saved in {save_dir}")

 Model and metadata saved in C:\users\LENOVO\Documents\formative2-mlp\Audios
